## Load Dataset

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from importlib import reload
import logging
reload(logging)
logging.basicConfig(format='%(asctime)s %(levelname)s:%(message)s', level=logging.INFO, datefmt='%H:%M:%S')

In [3]:
# libgomp issue, must import n2 before torch
from n2 import HnswIndex

In [4]:
import sys

sys.path.insert(0, '..')

In [5]:
import os
home_dir = os.getenv('HOME')

In [6]:
from collections import defaultdict
import itertools

def Enumerator(start=0, initial=()):
    return defaultdict(itertools.count(start).__next__, initial)

In [7]:
import glob
import csv
from tqdm.auto import tqdm

id_enumerator = Enumerator()
row_dict = {}
left_id_set = set()
right_id_set = set()
rows_total = 1081 + 1092
clusters_total = 1097

with tqdm(total=rows_total) as pbar:
    with open(f'{home_dir}/Downloads/Abt-Buy/Abt.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'abt'
            row_dict[row['id']] = row
            left_id_set.add(row['id'])
            pbar.update(1)
    
    with open(f'{home_dir}/Downloads/Abt-Buy/Buy.csv', encoding="latin1") as f:
        for row in csv.DictReader(f):
            row['id'] = id_enumerator[row["id"]]
            row['source'] = 'buy'
            row_dict[row['id']] = row
            right_id_set.add(row['id'])
            pbar.update(1)

In [8]:
true_pair_set = set()

with open(f'{home_dir}/Downloads/Abt-Buy/abt_buy_perfectMapping.csv') as f:
    for row in csv.DictReader(f):
        id_left = id_enumerator[row['idAbt']]
        id_right = id_enumerator[row['idBuy']]
        true_pair_set.add((id_left, id_right))

len(true_pair_set)

1097

In [9]:
from entity_embed.data_utils.utils import id_pairs_to_cluster_mapping_and_dict

cluster_mapping, cluster_dict = id_pairs_to_cluster_mapping_and_dict(true_pair_set)
len(cluster_mapping)

2173

In [10]:
len(cluster_dict)

1076

In [11]:
cluster_attr = 'cluster_id'
max_cluster_id = max(cluster_mapping.values())

for row_id, row in tqdm(row_dict.items()):
    try:
        row[cluster_attr] = cluster_mapping[row_id]
    except KeyError:
        row[cluster_attr] = max_cluster_id
        max_cluster_id += 1

In [12]:
[row_dict[row_id] for row_id in next(iter(true_pair_set))]

[{'id': 305,
  'name': 'TiVo HD Digital Video Recorder (180 Hour) - TCD652160',
  'description': 'TiVo HD Digital Video Recorder - TCD652160/ Search, Record And Watch Shows In HD/ Record Up To 20 Hours In HD (Or 180 Hours In Standard Definition)/ Record Two Shows At Once In HD/ Replaces Your Cable Box And Works With Over-The-Air Antenna/ USB Connectivity/ Remote Control/ Netflix Instant Streaming/ TiVo Service Required And Sold Separately',
  'price': '',
  'source': 'abt',
  'cluster_id': 305},
 {'id': 1508,
  'name': 'Tivo TIV652160 Tivo High Definition',
  'description': '',
  'manufacturer': 'TiVo',
  'price': '',
  'source': 'buy',
  'cluster_id': 305}]

## Preprocess

In [13]:
attr_list = ['name', 'description', 'price']

In [14]:
import unidecode
from entity_embed import default_tokenizer

def clean_str(s):
    s = unidecode.unidecode(s).lower().strip()
    s_tokens = itertools.islice((s_part[:30] for s_part in default_tokenizer(s)), 0, 30)
    return ' '.join(s_tokens)[:300]

for row in tqdm(row_dict.values()):
    for attr in attr_list:
        row[attr] = clean_str(row[attr])

## Init Data Module

In [15]:
import torch
import numpy as np

random_seed = 42
torch.manual_seed(random_seed)
np.random.seed(random_seed)

In [16]:
alphabet = list('0123456789abcdefghijklmnopqrstuvwxyz!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~ ')

In [21]:
attr_info_dict = {
    'name': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'description': {
        'field_type': "MULTITOKEN",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'use_mask': True,
        'max_str_len': None,  # compute
    },
    'price': {
        'field_type': "STRING",
        'tokenizer': "entity_embed.default_tokenizer",
        'alphabet': alphabet,
        'max_str_len': None,  # compute
    }
}

In [22]:
from entity_embed import AttrInfoDictParser

row_numericalizer = AttrInfoDictParser.from_dict(attr_info_dict, row_dict=row_dict)
row_numericalizer.attr_info_dict

10:42:13 INFO:For attr=name, computing actual max_str_len
10:42:13 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
10:42:13 INFO:For attr=name, using actual_max_str_len=16
10:42:13 INFO:For attr=description, computing actual max_str_len
10:42:13 INFO:actual_max_str_len=15 must be pair to enable NN pooling. Updating to 16
10:42:13 INFO:For attr=description, using actual_max_str_len=16
10:42:13 INFO:For attr=price, computing actual max_str_len
10:42:13 INFO:For attr=price, using actual_max_str_len=14


{'name': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', ':', ';', '<', '=', '>', '?', '@', '[', '\\', ']', '^', '_', '`', '{', '|', '}', '~', ' '], max_str_len=16, vocab=None, n_channels=8, embed_dropout_p=0.2, use_attention=True, use_mask=True),
 'description': NumericalizeInfo(field_type=<FieldType.MULTITOKEN: 'multitoken'>, tokenizer='entity_embed.data_utils.numericalizer.default_tokenizer', alphabet=['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',',

In [23]:
from entity_embed import LinkageDataModule

train_cluster_len = 200
valid_cluster_len = 200
datamodule = LinkageDataModule(
    row_dict=row_dict,
    cluster_attr=cluster_attr,
    row_numericalizer=row_numericalizer,
    batch_size=20,
    row_batch_size=16,
    train_cluster_len=train_cluster_len,
    valid_cluster_len=valid_cluster_len,
    test_cluster_len=clusters_total - valid_cluster_len - train_cluster_len,
    only_plural_clusters=True,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    random_seed=random_seed
)

## Training

In [24]:
from entity_embed import LinkageEmbed

ann_k = 100
model = LinkageEmbed(
    datamodule,
    ann_k=ann_k,
)

In [25]:
import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.loggers import TensorBoardLogger

max_epochs = 100
early_stop_callback = EarlyStopping(
   monitor='valid_recall_at_0.3',
   min_delta=0.00,
   patience=20,
   verbose=True,
   mode='max'
)
tb_log_dir = 'tb_logs'
tb_name = 'abt-buy'
trainer = pl.Trainer(
    gpus=1,
    max_epochs=max_epochs,
    check_val_every_n_epoch=1,
    callbacks=[early_stop_callback],
    logger=TensorBoardLogger(tb_log_dir, name=tb_name),
)

10:42:15 INFO:GPU available: True, used: True
10:42:15 INFO:TPU available: None, using: 0 TPU cores
10:42:15 INFO:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


In [26]:
trainer.fit(model, datamodule)

10:42:15 INFO:Train pair count: 214
10:42:15 INFO:Valid pair count: 200
10:42:15 INFO:Test pair count: 704
10:42:18 INFO:
  | Name        | Type       | Params
-------------------------------------------
0 | blocker_net | BlockerNet | 1.8 M 
1 | losser      | SupConLoss | 0     
-------------------------------------------
1.8 M     Trainable params
0         Non-trainable params
1.8 M     Total params


1

In [27]:
model.blocker_net.get_signature_weights()

{'name': 0.40336623787879944,
 'description': 0.3210184872150421,
 'price': 0.27561527490615845}

In [28]:
from entity_embed import validate_best

validate_best(trainer)

{'valid_f1_at_0.3': 0.1910507792860734,
 'valid_f1_at_0.5': 0.5685950413223141,
 'valid_f1_at_0.7': 0.6149425287356323,
 'valid_f1_at_0.9': 0.12206572769953053,
 'valid_pair_entity_ratio_at_0.3': 4.4725,
 'valid_pair_entity_ratio_at_0.5': 1.0125,
 'valid_pair_entity_ratio_at_0.7': 0.37,
 'valid_pair_entity_ratio_at_0.9': 0.0325,
 'valid_precision_at_0.3': 0.10620458356623812,
 'valid_precision_at_0.5': 0.4246913580246914,
 'valid_precision_at_0.7': 0.722972972972973,
 'valid_precision_at_0.9': 1.0,
 'valid_recall_at_0.3': 0.95,
 'valid_recall_at_0.5': 0.86,
 'valid_recall_at_0.7': 0.535,
 'valid_recall_at_0.9': 0.065}

## Testing

In [29]:
trainer.test(ckpt_path='best', verbose=False)

10:43:06 INFO:Train pair count: 214
10:43:06 INFO:Valid pair count: 200
10:43:06 INFO:Test pair count: 704


[{'test_f1_at_0.3': 0.06685563483530448,
  'test_f1_at_0.5': 0.3080413410637761,
  'test_f1_at_0.7': 0.4828897338403042,
  'test_f1_at_0.9': 0.09327846364883403,
  'test_pair_entity_ratio_at_0.3': 14.474377745241581,
  'test_pair_entity_ratio_at_0.5': 2.398975109809663,
  'test_pair_entity_ratio_at_0.7': 0.6500732064421669,
  'test_pair_entity_ratio_at_0.9': 0.02855051244509517,
  'test_precision_at_0.3': 0.034594375885090026,
  'test_precision_at_0.5': 0.1864510222764724,
  'test_precision_at_0.7': 0.42905405405405406,
  'test_precision_at_0.9': 0.8717948717948718,
  'test_recall_at_0.3': 0.991304347826087,
  'test_recall_at_0.5': 0.8855072463768116,
  'test_recall_at_0.7': 0.5521739130434783,
  'test_recall_at_0.9': 0.04927536231884058}]

## Testing manually

In [30]:
test_row_dict = datamodule.test_row_dict
test_left_vector_dict, test_right_vector_dict = model.predict(
    row_dict=test_row_dict,
    left_id_set=left_id_set,
    right_id_set=right_id_set,
    batch_size=16
)

In [31]:
embedding_size = model.blocker_net.embedding_size
test_true_pair_set = datamodule.test_true_pair_set

In [32]:
assert (len(test_left_vector_dict) + len(test_right_vector_dict)) == len(test_row_dict)

In [33]:
%%time

from entity_embed import ANNLinkageIndex

ann_index = ANNLinkageIndex(embedding_size=embedding_size)
ann_index.insert_vector_dict(left_vector_dict=test_left_vector_dict, right_vector_dict=test_right_vector_dict)
ann_index.build()

CPU times: user 404 ms, sys: 4.06 ms, total: 408 ms
Wall time: 65.2 ms


In [34]:
%%time

sim_threshold = 0.3
found_pair_set = ann_index.search_pairs(
    k=ann_k,
    sim_threshold=sim_threshold,
    left_vector_dict=test_left_vector_dict,
    right_vector_dict=test_right_vector_dict,
)

CPU times: user 534 ms, sys: 5.75 ms, total: 540 ms
Wall time: 71.5 ms


In [35]:
from entity_embed.evaluation import pair_entity_ratio

pair_entity_ratio(len(found_pair_set), len(test_row_dict))

14.474377745241581

In [36]:
from entity_embed.evaluation import precision_and_recall

precision_and_recall(found_pair_set, test_true_pair_set)

(0.034594375885090026, 0.991304347826087)

In [37]:
false_positives = list(found_pair_set - test_true_pair_set)
len(false_positives)

19088

In [38]:
false_negatives = list(test_true_pair_set - found_pair_set)
len(false_negatives)

6

In [39]:
cos_similarity = lambda a, b: np.dot(a, b)

In [40]:
for (id_left, id_right) in false_negatives[:10]:
    display(
        (
            cos_similarity(test_left_vector_dict[id_left], test_right_vector_dict[id_right]),
            row_dict[id_left], row_dict[id_right]
        )
    )

(0.29997003,
 {'id': 132,
  'name': "pioneer 6 . 5 ' 2 - way marine white speakers - tsmr1640",
  'description': "pioneer 6 . 5 ' 2 - way marine speakers - tsmr1640 / 160 watts maximum power handling ( 30 watts nominal )/ 1 - 1 / 8 ' poly",
  'price': '$ 120 . 00',
  'source': 'abt',
  'cluster_id': 939},
 {'id': 1232,
  'name': 'pioneer nautica ts - mr1640 marine speaker - ts - mr1640',
  'description': "pioneer electronics tsmr1640 nautica series 6 - 1 / 2 ' marine - use 2 - way speaker with 160 watts max . power",
  'manufacturer': 'Pioneer',
  'price': '',
  'source': 'buy',
  'cluster_id': 939})

(0.18175772,
 {'id': 343,
  'name': 'sony black locationfree base station - lfv30',
  'description': 'sony black locationfree base station - lfv30 / watch tv wherever you are / compatible with pc or psp system / programmable on - screen home remote / stream outside',
  'price': '$ 199 . 00',
  'source': 'abt',
  'cluster_id': 343},
 {'id': 1529,
  'name': 'sony lfv30 network media player',
  'description': 'ethernet , wi - fi',
  'manufacturer': 'Sony',
  'price': '$ 172 . 47',
  'source': 'buy',
  'cluster_id': 343})

(0.14869098,
 {'id': 1054,
  'name': 'griffin black tunebuds mobile for ipod and iphone - 9410tunbmblb',
  'description': 'griffin black tunebuds mobile for ipod and iphone - 9410tunbmblb / compatible with iphone and ipod / three sizes of cushioned silicone earpads / high - sensitivity microphone / controlmic',
  'price': '',
  'source': 'abt',
  'cluster_id': 1054},
 {'id': 1796,
  'name': 'griffin tunebuds stereo earset - 9410 - tunbmblb',
  'description': 'ear - bud',
  'manufacturer': 'Griffin',
  'price': '$ 19 . 53',
  'source': 'buy',
  'cluster_id': 1054})

(0.28818032,
 {'id': 186,
  'name': 'universal mrf - 350 rf black base station - mrf350',
  'description': 'universal mrf - 350 rf black base station - mrf350 / no more pointing / rf addressable / ir routing / expand operating range up to 100 feet / compatible',
  'price': '$ 250 . 00',
  'source': 'abt',
  'cluster_id': 186},
 {'id': 1437,
  'name': 'universal remote control mrf - 350 addressable narrow band rf base station with rfx - 250 antenna',
  'description': '',
  'manufacturer': 'UNIVERSAL REMOTE CONTROL, INC',
  'price': '$ 174 . 72',
  'source': 'buy',
  'cluster_id': 186})

(0.12881348,
 {'id': 298,
  'name': 'lg pearl gray xl capacity electric dryer - dle5955g',
  'description': 'lg pearl gray xl capacity electric dryer - dle5955g / 7 . 3 cu . ft . capacity / 9 drying programs / 5 temperature levels / precise temperature control',
  'price': '',
  'source': 'abt',
  'cluster_id': 298},
 {'id': 1503,
  'name': 'lg 7 . 3 cu . ft . front control , st drum , chrome door trim ( pearl gray )',
  'description': '',
  'manufacturer': 'LG Electronics',
  'price': '',
  'source': 'buy',
  'cluster_id': 298})

(0.24973519,
 {'id': 149,
  'name': 'waring professional cool - touch deep fryer - black / stainless steel finish - df100',
  'description': 'waring professional cool - touch deep fryer - df100 / large frying basket / 60 - minute timer / removable control panel / unique heating element / breakaway cord /',
  'price': '$ 70 . 00',
  'source': 'abt',
  'cluster_id': 149},
 {'id': 1351,
  'name': 'waring pro deep fryer 3qt - black',
  'description': '',
  'manufacturer': 'Waring',
  'price': '',
  'source': 'buy',
  'cluster_id': 149})